In [ ]:
# Do not use this code under normal circumstances and every time 
%pip install opencv-python matplotlib pillow

In [ ]:
!pip list # Do not use this code under normal circumstances and every time

In [ ]:
import tensorflow as tf # type: ignore
import os

In [ ]:
# Check GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs Available: ", len(gpus))
    print("GPU Details: ", gpus)
else:
    print("No GPUs Available.")

In [ ]:
# Check CPU availability
cpu = tf.config.list_physical_devices('CPU')
if gpus:
    print("CPUs Available: ", len(cpu))
    print("CPU Details: ", cpu)
else:
    print("No CPUs Available.")

In [ ]:
import cv2 # type: ignore
import imghdr
from matplotlib import pyplot as plt # type: ignore

In [ ]:
data_dir = r'training 4 classes\no Tumor - Tumor' #use data2 only if u are on this file

In [ ]:
os.listdir(data_dir)

In [ ]:
os.listdir(os.path.join(data_dir,'Tumor'))

In [ ]:
image_exts = ['jpeg','jpg','bmp','png']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
tf.data.Dataset??

In [ ]:
import numpy as np # type: ignore
from matplotlib import pyplot as plt # type: ignore

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir, batch_size=32) # the smaller batch size the better. the greater batch size the faster. the ideal batch size is 32

In [ ]:
# Assuming 'data' is your dataset
data_iterator = data.as_numpy_iterator()

In [ ]:
#get another batch from the iterator
batch = data_iterator.next()

In [ ]:
# images as numpy arrays
batch[0].shape # Should print something like (256, height, width, channels)

In [ ]:
#class 0 = class one no Brain Tumor 
#class 1 = class two Brain Tumor
batch[1] # Should print the labels for the 256 images

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
scaled = batch[0] / 255

In [ ]:
scaled.max()

In [ ]:
scaled.min()

In [ ]:
if batch[0].max() == 255:
    data = data.map(lambda x, y: (x / 255, y))
    batch = data.as_numpy_iterator().next()
    print('have been changed')
    print('batch[0].max() = ' + str(batch[0].max()))
    print('batch[0].min() = ' + str(batch[0].min()))
else:
    print('no changes')
    print('batch[0].max() = ' + str(batch[0].max()))
    print('batch[0].min() = ' + str(batch[0].min()))

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
batch = scaled_iterator.next()

In [ ]:
batch[0].max()

In [ ]:
batch[0].min()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.1)
test_size = int(len(data)*0.2)+1

In [ ]:
train_size + val_size + test_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

In [ ]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten # type: ignore

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from datetime import datetime

# Get the current date and time
current_date = datetime.now()

# Format the date string with microseconds
formatted_date = current_date.strftime('%d.%h.%Y') + '.' + current_date.strftime('(%H.%M.%S)') + '.' + str(current_date.microsecond)

print("Formatted date with microseconds:", formatted_date)

In [ ]:
logdir = fr'log_for_no Tumor - Tumor{str(formatted_date)}' # change it to any log file if u were working on this file

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
batch[0].max()

In [ ]:
batch[0].min()

In [ ]:
hist = model.fit(train, epochs=32, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
hist.history.keys()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="lower right") # 4 options: lower left, lower right, upper left, upper right
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper right") # 4 options: lower left, lower right, upper left, upper right
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy # type: ignore

In [ ]:
Precision = Precision()
Recall = Recall()
BinAccuracy = BinaryAccuracy()

In [ ]:
len(test)

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    Precision.update_state(y, yhat)
    Recall.update_state(y, yhat)
    BinAccuracy.update_state(y, yhat)

In [ ]:
print(f'Precision: {Precision.result().numpy()}, Recall {Recall.result().numpy()}, Accuracy{BinAccuracy.result().numpy()}')

In [ ]:
img = cv2.imread(r'training 4 classes\no Tumor - Tumor\AA no Tumor\image (16).jpg') # change this to image dir
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
np.expand_dims(resize, 0)

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
if yhat > 0.5 and yhat <= 1: 
    print(f'Image has been Classified as Brain Tumor')
elif yhat < 0.5 and yhat >= 0:
    print(f'Image has been Classified as no Brain Tumor')
elif yhat == 0.5:
    print(f'Neutral')
else:
    print(f'Bugged Code?!')

In [ ]:
from tensorflow.keras.models import load_model # type: ignore
from datetime import datetime
#import os
#import tensorflow as tf

In [ ]:
from datetime import datetime

num_epochs = len(hist.epoch)

# Get the current date and time
current_date = datetime.now()

# Format the date string with microseconds
formatted_date = current_date.strftime('%d.%h.%Y') + '.' + current_date.strftime('%H.%M.%S') + '.' + str(current_date.microsecond)

print("Formatted date with microseconds:", formatted_date)
print("len(hist.epochs) = " + str(num_epochs))

In [ ]:
model.save(os.path.join(r'models',f'Tumor{formatted_date}_{num_epochs}epochs.h5')) # choose the dir that u want to save ur model at

In [ ]:
new_model = load_model(os.path.join(r'models', fr'Tumor17.Jun.2024.18.44.33.777263_32epochs.h5')) # choose the dir that u want to load ur model from

In [ ]:
yhattest = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
if yhattest > 0.5 and yhattest <= 1: 
    print(f'Image has been Classified as Tumor')
elif yhattest < 0.5 and yhattest >= 0:
    print(f'Image has been Classified as no Tumor')
elif yhattest == 0.5:
    print(f'Neutral')
else:
    print(f'Bugged Code?!')

In [ ]:
import pandas as pd # type: ignore

# Extracting the metrics
epochs = range(1, len(hist.history['accuracy']) + 1)
accuracy = hist.history['accuracy']
validation_accuracy = hist.history['val_accuracy']
loss = hist.history['loss']
validation_loss = hist.history['val_loss']

# Creating a DataFrame with the desired columns
df = pd.DataFrame({
    'Epochs': epochs,
    'Accuracy': accuracy,
    'Validation Accuracy': validation_accuracy,
    'Loss': loss,
    'Validation Loss': validation_loss
})

# Saving the DataFrame to a CSV file
df.to_csv(rf'training_metrics_Tumor.csv', index=False)

In [ ]:
# Define the folder path
folder_path_Negative = r'testing/no tumor'
folder_path_Positive = r'testing/tumor'

In [ ]:
def predict_folder_brain_tumor(folder_path, model):
    # List all files in the folder
    files = os.listdir(folder_path)
    
    # Array to store names of images with incorrect predictions
    incorrect_predictions: list = []
    
    for file in files:
        # Construct full file path
        file_path = os.path.join(folder_path, file)
        
        # Read the image
        img = cv2.imread(file_path)
        if img is None:
            continue  # Skip if the file is not an image

        # Resize the image
        resize = tf.image.resize(img, (256, 256))

        # Prepare the image for prediction
        img_expanded = np.expand_dims(resize / 255.0, 0)

        # Make a prediction
        yhat = model.predict(img_expanded)

        # Interpret and print the prediction
        if yhat > 0.5 and yhat <= 1:
            print(f'Predicted class for {file} is Brain Tumor')
        elif yhat < 0.5 and yhat >= 0:
            print(f'Predicted class for {file} is No Brain Tumor')
            # Add incorrect prediction to the array
            incorrect_predictions.append(file)

    print(f'True Positive Percentage of Brain Tumor Prediction is: {float((len(files)-len(incorrect_predictions))/len(files)) * 100:.3f}%')

    
    # Return the array of incorrect predictions
    return incorrect_predictions


In [ ]:
def predict_folder_no_tumor(folder_path, model):
    # List all files in the folder
    files = os.listdir(folder_path)
    
    # Array to store names of images with incorrect predictions
    incorrect_predictions: list = []
    
    for file in files:
        # Construct full file path
        file_path = os.path.join(folder_path, file)
        
        # Read the image
        img = cv2.imread(file_path)
        if img is None:
            continue  # Skip if the file is not an image

        # Resize the image
        resize = tf.image.resize(img, (256, 256))

        # Prepare the image for prediction
        img_expanded = np.expand_dims(resize / 255.0, 0)

        # Make a prediction
        yhat = model.predict(img_expanded)

        # Interpret and print the prediction
        if yhat > 0.5 and yhat <= 1:
            print(f'Predicted class for {file} is Brain Tumor')
            # Add incorrect prediction to the array
            incorrect_predictions.append(file)
        elif yhat < 0.5 and yhat >= 0:
            print(f'Predicted class for {file} is No Brain Tumor')

    print(f'True Positive Percentage of no Brain Tumor Prediction is: {float((len(files)-len(incorrect_predictions))/len(files)) * 100:.3f}%')

    
    # Return the array of incorrect predictions
    return incorrect_predictions

In [ ]:
# Call the function to predict all images in the folder
predict_folder_brain_tumor(folder_path_Positive, new_model)

In [ ]:
# Call the function to predict all images in the folder
predict_folder_no_tumor(folder_path_Negative, new_model)